In [1]:
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

# X_train = torch.rand(350, 3, 3)
# Y_train = (torch.sum(X_train>0.5, dim=(1,2))>4).float()
# X_test = torch.rand(150, 3, 3)
# Y_test = (torch.sum(X_test>0.5, dim=(1,2))>4).float()

# data = {}
# data['x_train']=X_train.numpy().tolist()
# data['y_train']=Y_train.numpy().tolist()
# data['x_test']=X_test.numpy().tolist()
# data['y_test']=Y_test.numpy().tolist()

# with open('data.json', 'w') as fp:
#     json.dump(data, fp)

with open('data.json') as json_file:
    data = json.load(json_file)

X_train = torch.FloatTensor(data['x_train'])
Y_train = torch.FloatTensor(data['y_train'])
X_test = torch.FloatTensor(data['x_test'])
Y_test = torch.FloatTensor(data['y_test'])



150
350


In [6]:
print(X_train[:3])
print(Y_train[:3])

tensor([[[0.1053, 0.1793, 0.3774],
         [0.3989, 0.0384, 0.2504],
         [0.6553, 0.8287, 0.9172]],

        [[0.4858, 0.4974, 0.9633],
         [0.6753, 0.2577, 0.7881],
         [0.0708, 0.6123, 0.4570]],

        [[0.8439, 0.7446, 0.2441],
         [0.5449, 0.4565, 0.0235],
         [0.9623, 0.1675, 0.2396]]])
tensor([0., 0., 0.])


In [39]:
# create 3x3 tetrics data 
import numpy as np
import json
cords = {}
cords['S'] = np.array([
    [[1,1,0], [0,1,1], [0,0,0]], 
    [[0,0,0], [1,1,0], [0,1,1]], 
    [[0,1,0], [1,1,0], [1,0,0]], 
    [[0,0,1], [0,1,1], [0,1,0]],
    [[0,1,1], [1,1,0], [0,0,0]], 
    [[0,0,0], [0,1,1], [1,1,0]], 
    [[1,0,0], [1,1,0], [0,1,0]],
    [[0,1,0], [0,1,1], [0,0,1]],
])

cords['T'] = np.array([
    [[1,1,1], [0,1,0], [0,0,0]], 
    [[0,0,0], [1,1,1], [0,1,0]], 
    [[0,0,1], [0,1,1], [0,0,1]],
    [[0,1,0], [1,1,0], [0,1,0]],
    [[0,1,0], [1,1,1], [0,0,0]],
    [[0,0,0], [0,1,0], [1,1,1]],
    [[1,0,0], [1,1,0], [1,0,0]],
    [[0,1,0], [0,1,1], [0,1,0]],
])

cords['L'] = np.array([
    [[1,1,1], [0,0,1], [0,0,0]], 
    [[0,0,0], [1,1,1], [0,0,1]], 
    [[1,1,1], [1,0,0], [0,0,0]], 
    [[0,0,0], [1,1,1], [1,0,0]], 
    [[1,0,0], [1,1,1], [0,0,0]], 
    [[0,0,0], [1,0,0], [1,1,1]], 
    [[0,0,1], [1,1,1], [0,0,0]], 
    [[0,0,0], [0,0,1], [1,1,1]], 
    [[1,0,0], [1,0,0], [1,1,0]], 
    [[0,1,0], [0,1,0], [0,1,1]], 
    [[0,1,0], [0,1,0], [1,1,0]],
    [[0,0,1], [0,0,1], [0,1,1]],
    [[0,1,1], [0,1,0], [0,1,0]],
    [[1,1,0], [0,1,0], [0,1,0]],
    [[1,1,0], [1,0,0], [1,0,0]],
    [[0,1,1], [0,0,1], [0,0,1]],        
])

cords['O'] = np.array([
    [[1,1,0], [1,1,0], [0,0,0]], 
    [[0,1,1], [0,1,1], [0,0,0]], 
    [[0,0,0], [1,1,0], [1,1,0]],     
    [[0,0,0], [0,1,1], [0,1,1]], 
])

cords['I'] = np.array([
    [[1,0,0], [1,0,0], [1,0,0]], 
    [[0,1,0], [0,1,0], [0,1,0]], 
    [[0,0,1], [0,0,1], [0,0,1]],     
    [[1,1,1], [0,0,0], [0,0,0]], 
    [[0,0,0], [1,1,1], [0,0,0]], 
    [[0,0,0], [0,0,0], [1,1,1]], 
])

# print(np.concatenate((cords['S'],cords['S'])))

test_size = 40
train_size = 160

test = {}
train = {}

def create_sample(d, c, key, item):
    if key not in d.keys():
        d[key] = (np.random.random((1,3,3)) / 4 + 0.7) * item + np.random.random((1,3,3))/10
        return d
    d[key] = np.concatenate((d[key], (np.random.random((1,3,3)) / 4 + 0.7) * item + np.random.random((1,3,3))/10))
    return d



for key in cords.keys():
    count = 0
    ind = 0
    while count < test_size:
        item = cords[key][ind]
        test = create_sample(test, cords, key, item)
        count += 1
        ind += 1
        if ind==len(cords[key]):
            ind = 0

for key in cords.keys():
    count = 0
    ind = 0
    while count < train_size:
        item = cords[key][ind]
        train = create_sample(train, cords, key, item)
        count += 1
        ind += 1
        if ind==len(cords[key]):
            ind = 0

for key in cords.keys():
    train[key] = train[key].tolist()
    test[key] = test[key].tolist()

with open('tetris_train.json', 'w') as fp:
    json.dump(train, fp)

with open('tetris_test.json', 'w') as fp:
    json.dump(test, fp)

# print(train['S'])
# """
# S: 8
# T: 8
# L: 16
# I: 6
# O: 4
# """

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 3, 2)
        self.conv2 = nn.Conv2d(3, 3, 2)
#         self.conv1 = nn.Conv2d(1, 2, 3, padding=1)
#         self.mp1 = nn.MaxPool2d(3, stride=1, padding=1)
#         self.conv2 = nn.Conv2d(2, 3, 3, padding=1)
#         self.mp2 = nn.MaxPool2d(3, stride=1, padding=1)
#         self.mp = nn.MaxPool2d(2, stride=1)
        self.fc = nn.Linear(3, 1)

    def forward(self, x):
        in_size = x.size(0)
#         x = self.mp1(F.relu(self.conv1(x)))
#         x = self.mp2(F.relu(self.conv2(x)))
        
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        
        x = x.view(in_size, -1)
        x = self.fc(x)
        return x

model = ConvNet()

criterion = nn.MSELoss()
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.01)

#
# Print number of parameters
#
for item in list(ConvNet().parameters()):
    print (item)
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

get_n_params(ConvNet())

In [ ]:
# 
# Training
# 
results = {} #{iter: [loss, accuracy]}
for epoch in range(1000):  # loop over the dataset multiple times

    running_loss = 0.0
    for i in range(len(Y_train)):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = X_train[i], torch.tensor([Y_train[i]])
        inputs = inputs.view(1,1,len(inputs), len(inputs))
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        # print(outputs.squeeze(0), labels)
        loss = criterion(outputs.squeeze(0), labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    
    if epoch % 10 == 9:
        print ("Epoch: %d, Loss: %.3f"%(epoch+1, running_loss))

        correct = 0.
        total = float(len(Y_test))

        with torch.no_grad():
            for i in range(len(Y_test)):
                images, labels = X_test[i], torch.tensor(Y_test[i])
                images = images.view(1,1,len(images), len(images))

                output = model(images)
                predicted = output.data > 0.5
                correct += (int(predicted[0]) == int(labels))
                accuracy = 100. * correct / total
        print('Accuracy of the network: %.2f %%' % (accuracy))
        
        results[epoch+1] = [running_loss, accuracy]

In [ ]:
with open('results.json', 'w') as fp:
    json.dump(results, fp)
    

# correct = 0.
# total = float(len(Y_test))

# outputs = []
# with torch.no_grad():
#     for i in range(len(Y_test)):
#         images, labels = X_test[i], torch.tensor(Y_test[i])
#         images = images.view(1,1,len(images), len(images))
        
#         output = model(images)
#         outputs.append(output)

#         predicted = output.data > 0.5

#         correct += (int(predicted[0]) == int(labels))

# print('Accuracy of the network on the %d test images: %d %%' % (len(Y_test),
#     100 * correct / total))